In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
from matplotlib import pyplot as plt
linkage_name = '20190114_002018_meshID_444481be-5cba-4dad-a316-b52203791b61'
l = elastic_rods.RodLinkage('../examples/20190114_002018_meshID_444481be-5cba-4dad-a316-b52203791b61.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [15, 3])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

In [ ]:
view = LinkageViewer(l, width=1024)
view.show()

In [ ]:
import deployment_path_analysis
dpa = deployment_path_analysis.deploymentPathAnalysis(l)
deployment_path_analysis.validateEnergyIncrements(l, epsMax=1e-2)

In [ ]:
openingAngles, stiffnessGaps = deployment_path_analysis.stiffnessGapThroughoutDeployment(l, np.deg2rad(62), 100)
view.update()
plt.plot(openingAngles, stiffnessGaps)
plt.ylabel('Relative Deployment Stiffness Gap')
plt.xlabel('Opening Angle')
plt.show()

In [ ]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [ ]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [ ]:
from open_linkage import open_linkage
elastic_rods.benchmark_reset()
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-12
    opts.useIdentityMetric = False
    opts.niter = 1000
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
with suppress_stdout(): cr = open_linkage(l, driver, np.deg2rad(85) - l.averageJointAngle, 100, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=80, verbose=10, useTargetAngleConstraint=True)
elastic_rods.benchmark_report()

## Stiffness along the deployment path
We actually see negative stiffness midway through the deployment, which explains the numerical problems during opening and inaccuracies in the shape derivative quantities in the deployed state (if the structure is insufficiently stretched).

In [ ]:
angles = cr[2][:-1]
energies = [r.energy[-1] for r in cr[0]][:-1]
plt.plot(angles, energies)
plt.show()

In [ ]:
plt.plot(angles, np.gradient(np.array(energies), edge_order=2))
plt.show()

In [ ]:
plt.plot(angles, np.gradient(np.gradient(np.array(energies), edge_order=2), edge_order=2))
plt.ylim([-5, 10])
plt.grid()
plt.show()

In [ ]:
openingAngles = cr[2]

In [ ]:
energies = np.array([r.energy[-1] for r in cr[0]])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(openingAngles, energies)

In [ ]:
import compute_vibrational_modes
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0)
mview.show()

In [ ]:
import compute_vibrational_modes
fixedVarsWithoutActuator = fixedVars[:-1]
# lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=fixedVarsWithoutActuator, mtype=compute_vibrational_modes.MassMatrixType.FULL)
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=15)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0)
mview.show()

In [ ]:
from write_render_files import writeRenderFiles
writeRenderFiles(l, 'Boat', 'the_boat_depl')